# 机器人道路跟踪-模型训练
在这个示例中，我们将采用ResNet18神经网络结构来训练模型。
> “ResNet18”相比“alexnet”的网络结构会更加复杂，但是训练出来的模型表现力会更好。
## 1.导入所需模块

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 2.加载数据
1. 解析图片名称；
> 在保存数据时，我们给每张图片都添加了关于 x,y的位置信息，因此我们需要把x,y的信息提取出来
2. 自己定义一个类，用于管理我们的数据并且给数据做预处理。

In [ ]:
 # 解析x的信息
def get_x(path):
    return (float(int(path[3:6])) - 50.0) / 50.0

# 解析y的信息
def get_y(path):
    return (float(int(path[7:10])) - 50.0) / 50.0

# 定义XYDataset类，来管理我们的数据。
class XYDataset(torch.utils.data.Dataset):
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
        
    # 计算图片数量
    def __len__(self):
        return len(self.image_paths)
    
    # 读取图片
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        # 随机翻转图片
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        # 改变图片对比度，亮度，增加图片数量
        image = self.color_jitter(image)
        # resnet18输入的图片尺寸为224×224，因此需要把图片变换成这个尺寸
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        # 预处理图片
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

## 3.将数据集分割为训练集和测试集

我们把数据集的90%进行训练，10%的数据进行测试。测试集将用于验证我们训练的模型的准确性。

In [ ]:
test_percent = 0.2
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

## 4.创建数据加载器

我们使用“DataLoader”模块进行批量加载数据到神经网络。我们打乱所有的图片顺序，每次随机的从数据中提取64张图片作为一批输入神经网络，批量大小取决于GPU的性能。“num_workers=4”表示我们加载数据时使用的核心数量。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4
)

## 5.定义神经网络模型

和“避障”实验一样，我们也会用到迁移学习，提高训练效率和模型准确率，我们将加载“resnet18”网络结构和预训练模型。

In [ ]:
model = models.resnet18(pretrained=True)

## 6.改变网络输出神经元
这里和“alexnet”有点不一样，“resnet”的最后一层输入为512个特征，因为我们只要2个坐标的回归值，所以需要把输出数量改为2。
最后，我们将模型转移到GPU上执行

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

## 7.回归模型训练:

我们将训练50个epoch，每训练完一个epoch会分别打印出loss在训练数据集上和在测试数据集上的结果，最终我们会保存在测试集上loss最小的模型，因为在测试集上的loss更能反应出在真实场景下模型的表现力的强弱。训练完成后可以在左边目录中看到'best_steering_model_xy.pth'模型文件。

In [ ]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'studens_models/best_steering_model_xy.pth'
best_loss = 1e9

# adam自适应优化器
optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    # 模型训练
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    # 模型评估
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    # 打印训练和评估时loss的结果
    print('%f, %f' % (train_loss, test_loss))
    
    # 保存测试时loss最低的模型
    if test_loss < best_loss:
        torch.save(model, BEST_MODEL_PATH)
        best_loss = test_loss
print('训练完成！')

当我们训练完成后，您可以使用这个模型在“livedemo”中进行实验了。
你也可以在电脑上进行训练，完成之后把模型上传到这个目录下面。